In [5]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Add, Input, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.optimizers import Adam
import numpy as np
import imgaug as ia
import imgaug.augmenters as iaa

# Model Parameters
pixel_width = 28
pixel_height = 28
input_shape = (pixel_width, pixel_height, 1)
num_of_classes = 10
batch_size = 64
epochs = 25  # Increased epochs
learning_rate = 0.001
dropout_rate = 0.3  # Reduced dropout slightly
l2_reg = 0.0001 # reduced l2 slightly.

# Load MNIST Dataset
(features_train, labels_train), (features_test, labels_test) = keras.datasets.mnist.load_data()

# Reshape and Normalize Data
features_train = features_train.reshape(-1, pixel_width, pixel_height, 1).astype("float32") / 255.0
features_test = features_test.reshape(-1, pixel_width, pixel_height, 1).astype("float32") / 255.0

# One-hot encode labels
labels_train = keras.utils.to_categorical(labels_train, num_of_classes)
labels_test = keras.utils.to_categorical(labels_test, num_of_classes)

def elastic_transform_augmentation(images):
    """Applies elastic transformations using imgaug."""
    # Ensure images are a numpy array
    images = np.array(images)

    # Remove the channel dimension to make it (N, 28, 28)
    images = np.squeeze(images, axis=-1)

    seq = iaa.Sequential([
        iaa.ElasticTransformation(alpha=(25, 50), sigma=5.0)  # Adjust alpha and sigma as needed
    ])
    #imgaug expects a list of images.
    augmented_images = seq(images=images)

    # Add the channel dimension back
    augmented_images = np.expand_dims(np.array(augmented_images), axis=-1)

    return augmented_images
    
# Data Augmentation (Expanded)
datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.25,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    preprocessing_function=elastic_transform_augmentation,
)

datagen.fit(features_train)

# Residual Block Function
def residual_block(x, filters, kernel_size=3):
    shortcut = x
    x = Conv2D(filters, kernel_size, padding='same', activation='relu', kernel_regularizer=l2(l2_reg))(x)
    x = BatchNormalization()(x)
    x = Conv2D(filters, kernel_size, padding='same', kernel_regularizer=l2(l2_reg))(x)
    x = BatchNormalization()(x)
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

# Enhanced Model with Residual Blocks
inputs = Input(shape=input_shape)
x = Conv2D(32, 3, padding='same', activation='relu', kernel_regularizer=l2(l2_reg))(inputs)
x = BatchNormalization()(x)
x = residual_block(x, 32)
x = MaxPooling2D(2)(x)
x = Dropout(dropout_rate)(x)

x = Conv2D(64, 3, padding='same', activation='relu', kernel_regularizer=l2(l2_reg))(x)
x = BatchNormalization()(x)
x = residual_block(x, 64)
x = MaxPooling2D(2)(x)
x = Dropout(dropout_rate)(x)

x = Flatten()(x)
x = Dense(256, activation='relu', kernel_regularizer=l2(l2_reg))(x)
x = BatchNormalization()(x)
x = Dropout(dropout_rate)(x)
outputs = Dense(num_of_classes, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

# Cosine Annealing Learning Rate Schedule
def lr_schedule(epoch, lr):
    decay_steps = epochs
    cosine_decay = 0.5 * (1 + np.cos(np.pi * epoch / decay_steps))
    decayed = (1 - 1e-4) * cosine_decay + 1e-4
    return lr * decayed

lr_callback = LearningRateScheduler(lr_schedule)

# Optimizer and Compilation
optimizer = Adam(learning_rate=learning_rate)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7, verbose=1)

# Training with Augmented Data
model.fit(datagen.flow(features_train, labels_train, batch_size=batch_size),
          validation_data=(features_test, labels_test),
          epochs=epochs,
          verbose=1,
          callbacks=[early_stopping, reduce_lr, lr_callback])

# Evaluation
score = model.evaluate(features_test, labels_test, verbose=0)
print(f"Final Loss: {score[0]:.5f}, Accuracy: {score[1] * 100:.2f}%")

# Save the trained model
model.save('model_advanced.h5')
print("Model saved as 'model_advanced.h5'")

Epoch 1/25
938/938 ━━━━━━━━━━━━━━━━━━━━ 174s 181ms/step - accuracy: 0.5395 - loss: 1.5329 - val_accuracy: 0.9636 - val_loss: 0.1993 - learning_rate: 0.0010
Epoch 2/25
938/938 ━━━━━━━━━━━━━━━━━━━━ 165s 176ms/step - accuracy: 0.8374 - loss: 0.6024 - val_accuracy: 0.9516 - val_loss: 0.2454 - learning_rate: 9.9606e-04
Epoch 3/25
938/938 ━━━━━━━━━━━━━━━━━━━━ 164s 175ms/step - accuracy: 0.8734 - loss: 0.5052 - val_accuracy: 0.9760 - val_loss: 0.1801 - learning_rate: 9.8041e-04
Epoch 4/25
938/938 ━━━━━━━━━━━━━━━━━━━━ 177s 189ms/step - accuracy: 0.8912 - loss: 0.4529 - val_accuracy: 0.9779 - val_loss: 0.1749 - learning_rate: 9.4599e-04
Epoch 5/25
938/938 ━━━━━━━━━━━━━━━━━━━━ 173s 184ms/step - accuracy: 0.9005 - loss: 0.4271 - val_accuracy: 0.9795 - val_loss: 0.1769 - learning_rate: 8.8749e-04
Epoch 6/25
938/938 ━━━━━━━━━━━━━━━━━━━━ 182s 194ms/step - accuracy: 0.9120 - loss: 0.4021 - val_accuracy: 0.9629 - val_loss: 0.2324 - learning_rate: 8.0275e-04
Epoch 7/25
938/938 ━━━━━━━━━━━━━━━━━━━━ 0s 1

Final Loss: 0.14836, Accuracy: 97.87%


AttributeError: module 'h5py' has no attribute 'File'